In [1]:
import sys
import json
import io

In [2]:
!{sys.executable} -V

Python 3.9.6


In [3]:
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
from google.cloud import bigquery
from google.cloud.bigquery import Table

from googleapiclient import discovery
from oauth2client.client import GoogleCredentials

In [5]:
project_id = 'bigquery-project-32089'
dataset_id = 'customers_dataset'
table_id = 'customers'
dataset_full_name = f"{project_id}.{dataset_id}"
table_full_name = f"{project_id}.{dataset_id}.{table_id}"

In [6]:
client = bigquery.Client(project=project_id)

In [7]:
def list_datasets():
    datasets = list(client.list_datasets())
    project = client.project

    if datasets:
        print("Datasets in project {}:".format(project))
        for dataset in datasets:
            print("\t{}".format(dataset.dataset_id))
    else:
        print("{} project does not contain any datasets.".format(project))

In [8]:
# List available datasets
list_datasets()

Datasets in project bigquery-project-32089:
	customers_dataset


## 1.) Load customers_1.json to create a table in Python and load the data into it.

In [9]:
#  create a table in Python and load the data into it.
def create_table(tablefullname):    
    table_id = Table.from_string(tablefullname)
    table = bigquery.Table(table_id)
    
    table = client.create_table(table)
    
    print(
        "Created table {}.{}.{}".format(table.project, table.dataset_id, table.table_id)
    )

In [10]:
create_table(table_full_name)

Created table bigquery-project-32089.customers_dataset.customers


In [11]:
# Load customers_1.json file
customers_1_json_file = '.\data\customers_1.json'
customers_1_json_data = [json.loads(line) for line in open(customers_1_json_file, 'r')]

### This function will infer and update schemas without specifying them manually.

In [12]:
# This function will infer and update schemas without specifying them manually from given json file.

def load_json_with_auto_detect_schema(project_id,dataset_id,table_id,customers_1_json):   
    project = client.project
    dataset_ref = bigquery.DatasetReference(project, dataset_id)    

    # Retrieves the destination table and checks the length of the schema    
    table_ref = dataset_ref.table(table_id)
    table = client.get_table(table_ref)
    print("Table {} contains {} columns.".format(table_id, len(table.schema)))    
    
    job_config = bigquery.LoadJobConfig(
        autodetect=True,
        source_format=bigquery.SourceFormat.NEWLINE_DELIMITED_JSON
    )     
        
    job = client.load_table_from_json(customers_1_json, table_ref, location="US", job_config=job_config)
    
    job.result()  # Waits for table load to complete.
    
    print(
        "Loaded {} rows into {}:{}.".format(
            job.output_rows, dataset_id, table_ref.table_id
        )
    )

    # Checks the updated length of the schema
    table = client.get_table(table)
    print("Table {} now contains {} columns.".format(table_id, len(table.schema)))

In [13]:
# Update table schema with new fields
load_json_with_auto_detect_schema(project_id,dataset_id,table_id,customers_1_json_data)

Table customers contains 0 columns.
Loaded 4 rows into customers_dataset:customers.
Table customers now contains 20 columns.


### 2.) Load customers_2.json into the table using a load job - update the schema of table without manually specifying the names of the new fields in the customers_2.json

In [14]:
# Load customers_2.json file has additional fields. Table needs to be updated for the schema change.
customers_2_json_file = '.\data\customers_2.json'
customers_2_json_data = [json.loads(line) for line in open(customers_2_json_file, 'r')]

In [15]:
# This function will update the schema of table without manually specifying the names of the new fields

def update_schema_insert_fields(project_id,dataset_id,table_id,customers_2_json):   
    project = client.project
    dataset_ref = bigquery.DatasetReference(project, dataset_id)    

    # Retrieves the destination table and checks the length of the schema    
    table_ref = dataset_ref.table(table_id)
    table = client.get_table(table_ref)
    print("Table {} contains {} columns.".format(table_id, len(table.schema)))
    
    
    job_config = bigquery.LoadJobConfig(
        autodetect=True,
        source_format=bigquery.SourceFormat.NEWLINE_DELIMITED_JSON,
        schema_update_options=[
            bigquery.SchemaUpdateOption.ALLOW_FIELD_ADDITION,
            bigquery.SchemaUpdateOption.ALLOW_FIELD_RELAXATION
        ],
        write_disposition=bigquery.WriteDisposition.WRITE_APPEND
    ) 
        
    job = client.load_table_from_json(customers_2_json, table_ref, location="US", job_config=job_config)
    
    job.result()  # Waits for table load to complete.
    print(
        "Loaded {} rows into {}:{}.".format(
            job.output_rows, dataset_id, table_ref.table_id
        )
    )

    # Checks the updated length of the schema
    table = client.get_table(table)
    print("Table {} now contains {} columns.".format(table_id, len(table.schema)))

In [16]:
# Update table schema with new fields
update_schema_insert_fields(project_id,dataset_id,table_id,customers_2_json_data)

Table customers contains 20 columns.
Loaded 4 rows into customers_dataset:customers.
Table customers now contains 22 columns.


# Implementing Bigquery table with PATCH method

### This is a REST API , so we need service discovery 
### https://cloud.google.com/bigquery/docs/reference/rest/v2/tables/patch

In [17]:
#!{sys.executable} -m pip install google-api-python-client

In [18]:
#!{sys.executable} -m pip install --upgrade oauth2client

In [19]:
project = client.project
dataset_ref = bigquery.DatasetReference(project, dataset_id) 
table_ref = dataset_ref.table(table_id)
table = client.get_table(table_ref)

In [20]:
#to get credentials from my laptop
credentials = GoogleCredentials.get_application_default()
# Construct the service object for interacting with the BigQuery API.
bq = discovery.build('bigquery', 'v2', credentials=credentials)

In [21]:
credentials

In [22]:
bq

### View current table's schema as a JSON , this is needs for the comparison with new schema to be submitted with new Fields and their Types.

In [23]:
f = io.StringIO("")
client.schema_to_json(table.schema, f)
#client.schema_from_json
print(f.getvalue())

[
  {
    "mode": "NULLABLE",
    "name": "favoriteFruit",
    "type": "STRING"
  },
  {
    "fields": [
      {
        "mode": "NULLABLE",
        "name": "name",
        "type": "STRING"
      },
      {
        "mode": "NULLABLE",
        "name": "id",
        "type": "INTEGER"
      }
    ],
    "mode": "REPEATED",
    "name": "friends",
    "type": "RECORD"
  },
  {
    "mode": "NULLABLE",
    "name": "latitude",
    "type": "FLOAT"
  },
  {
    "description": "bq-datetime",
    "mode": "NULLABLE",
    "name": "registered",
    "type": "TIMESTAMP"
  },
  {
    "mode": "NULLABLE",
    "name": "address",
    "type": "STRING"
  },
  {
    "mode": "NULLABLE",
    "name": "longitude",
    "type": "FLOAT"
  },
  {
    "mode": "NULLABLE",
    "name": "phone",
    "type": "STRING"
  },
  {
    "mode": "NULLABLE",
    "name": "greeting",
    "type": "STRING"
  },
  {
    "mode": "NULLABLE",
    "name": "email",
    "type": "STRING"
  },
  {
    "mode": "REPEATED",
    "name": "tags",
    

In [24]:
# Load customers_3.json file that includes another schema change.
customers_3_json_file = r'.\data\customers_3.json'

## Prepared the new schema accordingly & viewing with method "schema_from_json" for the schema comparison.
## This has been done manually.

In [25]:
customer_3_schema_file_path = '.\customers_3_schema.json'
schema1 = client.schema_from_json(customer_3_schema_file_path)
schema1

[SchemaField('_id', 'STRING', 'NULLABLE', None, (), None),
 SchemaField('index', 'INTEGER', 'NULLABLE', None, (), None),
 SchemaField('guid', 'STRING', 'NULLABLE', None, (), None),
 SchemaField('isActive', 'BOOLEAN', 'NULLABLE', None, (), None),
 SchemaField('balance', 'FLOAT', 'NULLABLE', None, (), None),
 SchemaField('picture', 'STRING', 'NULLABLE', None, (), None),
 SchemaField('age', 'INTEGER', 'NULLABLE', None, (), None),
 SchemaField('eyeColor', 'STRING', 'NULLABLE', None, (), None),
 SchemaField('name', 'STRING', 'NULLABLE', None, (), None),
 SchemaField('gender', 'STRING', 'NULLABLE', None, (), None),
 SchemaField('company', 'STRING', 'NULLABLE', None, (), None),
 SchemaField('email', 'STRING', 'NULLABLE', None, (), None),
 SchemaField('phone', 'STRING', 'NULLABLE', None, (), None),
 SchemaField('address', 'STRING', 'NULLABLE', None, (), None),
 SchemaField('about', 'STRING', 'NULLABLE', None, (), None),
 SchemaField('registered', 'TIMESTAMP', 'NULLABLE', None, (), None),
 Sche

## But, we need flatten schema without "SchemaField"

In [26]:
with open(customer_3_schema_file_path) as f:
    customer_3_schema = json.load(f)
f.close()

In [27]:
customer_3_schema

[{'name': '_id', 'type': 'STRING', 'mode': 'NULLABLE'},
 {'name': 'index', 'type': 'INTEGER', 'mode': 'NULLABLE'},
 {'name': 'guid', 'type': 'STRING', 'mode': 'NULLABLE'},
 {'name': 'isActive', 'type': 'BOOLEAN', 'mode': 'NULLABLE'},
 {'name': 'balance', 'type': 'FLOAT', 'mode': 'NULLABLE'},
 {'name': 'picture', 'type': 'STRING', 'mode': 'NULLABLE'},
 {'name': 'age', 'type': 'INTEGER', 'mode': 'NULLABLE'},
 {'name': 'eyeColor', 'type': 'STRING', 'mode': 'NULLABLE'},
 {'name': 'name', 'type': 'STRING', 'mode': 'NULLABLE'},
 {'name': 'gender', 'type': 'STRING', 'mode': 'NULLABLE'},
 {'name': 'company', 'type': 'STRING', 'mode': 'NULLABLE'},
 {'name': 'email', 'type': 'STRING', 'mode': 'NULLABLE'},
 {'name': 'phone', 'type': 'STRING', 'mode': 'NULLABLE'},
 {'name': 'address', 'type': 'STRING', 'mode': 'NULLABLE'},
 {'name': 'about', 'type': 'STRING', 'mode': 'NULLABLE'},
 {'name': 'registered', 'type': 'TIMESTAMP', 'mode': 'NULLABLE'},
 {'name': 'latitude', 'type': 'FLOAT', 'mode': 'NULLA

## Updating Bigquery table with PATCH method
##### https://cloud.google.com/bigquery/docs/reference/rest/v2/tables/patch

In [28]:
tables = bq.tables()
table_reference = {"projectId":project_id, "tableId":table_id, "datasetId": dataset_id} 
table_reference = {"tableReference": table_reference,"schema": { "fields": customer_3_schema}} 
table_reference.update(table_reference)

tableStatusObject =tables.patch(projectId=project_id,datasetId=dataset_id,tableId=table_id, 
body=table_reference,autodetect_schema=True).execute()
print("BQ Table Patched with customer_3_schema_fields ..!!")

BQ Table Patched with customer_3_schema_fields ..!!
